In [1]:
import math
import pandas as pd

In [2]:
with open(r'Data\reviews_sample.txt','r') as f:
    content=f.readlines()
content = [x.strip() for x in content] 
content = [x.split(' ') for x in content]

#Initialize minimum support
minsup_pct=0.01
minsup = math.floor(len(content)*0.01)

#write the transaction database into vertical database format
li = [(i,j,content[i][j],1) for i in range(len(content)) for j in range(len(content[i]))]
df=pd.DataFrame(data=li, columns=['SID','EID','pattern','pattern_len']) #transaction database in vertical format, also candidate 1 set

In [3]:
def get_candidate_k(k):
    tmp_df=pd.merge(df[(df.pattern_len==k-1) & (df.pattern.isin(df_frequent.index))] , df[(df.pattern_len==k-1) & (df.pattern.isin(df_frequent.index))], on=['SID','pattern_len'], suffixes=['_x','_y'])
    
    #Phrase mining, to only min consecutive frequent sequence
    tmp_df=tmp_df[tmp_df['EID_y']==(tmp_df['EID_x']+1)]
    
    #generate candidate length k phrase
    tmp_df[['pattern','pattern_len']]=tmp_df.apply(lambda row: merge_pattern(row['pattern_x'], row['pattern_y'], row['pattern_len']), axis=1)
    
    #clean up the candidate k set dataframe
    tmp_df=tmp_df.rename(columns={'EID_x':'EID'})
    tmp_df.drop(['EID_y','pattern_x','pattern_y'], axis=1, inplace=True)
    return tmp_df

In [4]:
def get_frequent_k(k):
    tmp_df_frequent=df[df.pattern_len==k].groupby('pattern').SID.nunique()[df[df.pattern_len==k].groupby('pattern').SID.nunique() >= minsup]
    return tmp_df_frequent

In [5]:
def merge_pattern(pattern1, pattern2, pattern_len):
    pattern2=pattern2[pattern2.rfind('|')+1:]
    return pd.Series([pattern1+'|'+pattern2, pattern_len+1])

In [6]:
k=1
while True:
    if k == 1:
        #retrieve frequent 1 item set and count
        df_frequent=df.groupby('pattern').SID.nunique()[df.groupby('pattern').SID.nunique() >= minsup]
    else:
        #append length k candidate phrase to candidate dataframe 
        tmp_df=get_candidate_k(k)
        df=df.append(tmp_df)
        #append length k frequent phrase to frequent series
        tmp_df_frequent=get_frequent_k(k)
        df_frequent=df_frequent.append(tmp_df_frequent)
        
        if len(tmp_df_frequent)==0:
            break       
    k+=1

In [20]:
file  = open('Output/patterns.txt', 'w')
for x in df_frequent:
    line=str(x[1])+':'
    if str(type(x[0])).find('str') != -1:
        line=line+x[0]+';'
    else:
        for i in x[0]:
            line=line+i+';'
    file.write(line[:-1]+'\n')
    print(line)

121:100;
402:able;
392:absolutely;
261:across;
570:actually;
204:add;
163:added;
112:addition;
191:afternoon;
314:ago;
127:ahead;
436:almost;
129:alone;
193:along;
241:already;
1985:also;
356:although;
1475:always;
839:amazing;
148:ambiance;
125:american;
286:amount;
756:another;
277:anyone;
582:anything;
176:anyway;
207:anywhere;
131:apparently;
466:appetizer;
125:apple;
162:appointment;
125:appreciate;
867:area;
221:aren;
1005:around;
261:arrived;
148:art;
450:ask;
620:asked;
130:asking;
264:ate;
625:atmosphere;
114:attention;
238:attentive;
108:attitude;
158:authentic;
228:available;
297:average;
123:avoid;
617:away;
638:awesome;
148:awful;
2194:back;
197:bacon;
845:bad;
137:bag;
137:baked;
122:bakery;
1211:bar;
100:barely;
294:bartender;
202:based;
113:basic;
156:basically;
178:bathroom;
253:bean;
149:beat;
299:beautiful;
100:become;
140:bed;
257:beef;
897:beer;
197:behind;
278:believe;
1721:best;
1194:better;
120:beyond;
738:big;
214:bill;
195:birthday;
831:bit;
250:bite;
158:blac